In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU340"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso Salvare - Rescue Dogs')

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Personal Logo
image_filename2 = 'MB_logo_LG.png'
encoded_image2 = base64.b64encode(open(image_filename2, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # top container to hold Grazioso Salvare's logo and Dash Board title
    html.Div(
        style={'display': 'flex', 'align-items': 'flex-end'},
        children=[
            # Display Logo as a clickable link    
            html.A(
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                style={'width': '150px', 'height': 'auto', 'margin-right': '40px'}),
                href='https://www.snhu.edu', target='_blank'
            ),
            html.Center(
                html.B(html.H1('List of Animals in Shelters')),
                style={'margin-right': '350px'}
            ),
        ]
    ),
    html.Hr(),
    # Radio buttons utilized to filter the displayed data in the data table
    html.Div(className='radioItems',
             style= {'display' : 'flex'},
                 children =[
                    dcc.RadioItems(
                        options=[
                            {'label': 'Water Rescue', 'value': 'Water'},
                            {'label': 'Mountain or Wilderness Rescue', 'value': 'MountWilderness'},
                            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
                            {'label': 'Reset', 'value': 'Reset'},
                        ],
                        id='animal-type-checklist',
                        value='Reset',  # Initially select 'Reset' by default
                        inline=True,
                        style={'display': 'flex'},
                        labelStyle={'padding-right': '20px'}
                    )
                 ]),
    html.Hr(),
    
    # Container for the data table and it's styling and a sunburst graph wigdet that will
    # display all available trainable dogs based on the filtering 
    # puts data table on left side of screen with graph on the right side in a row
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            dash_table.DataTable(id='datatable-id',
                columns=[
                    {"name": "Animal Type", "id": "animal_type", "deletable": False, "selectable": True},
                    {"name": "Name", "id": "name", "deletable": False, "selectable": True},
                    {"name": "Breed", "id": "breed", "deletable": False, "selectable": True},
                    {"name": "Color", "id": "color", "deletable": False, "selectable": True},
                    {"name": "Sex", "id": "sex_upon_outcome", "deletable": False, "selectable": True},
                    {"name": "Age", "id": "age_upon_outcome", "deletable": False, "selectable": True},
                    {"name": "Date of birth", "id": "date_of_birth", "deletable": False, "selectable": True},
                    {"name": "ID number", "id": "animal_id", "deletable": False, "selectable": True},
                    {"name": "Requisition Number", "id": "rec_num", "deletable": False, "selectable": True},
                    {"name": "Acquisition Type", "id": "outcome_type", "deletable": False, "selectable": True},
                    {"name": "Notes", "id": "outcome_subtype", "deletable": False, "selectable": True},
                    {"name": "Location Latitude", "id": "location_lat", "deletable": False, "selectable": True},
                    {"name": "Location Longitude", "id": "location_long", "deletable": False, "selectable": True},
                    {"name": "Age in weeks", "id": "age_upon_outcome_in_weeks", "deletable": False, "selectable": True}
                ],
                data=df.to_dict('records'),
                style_cell={'textAlign': 'left', 'padding-right': '10px'},
                editable = False,
                filter_action = "native",
                sort_action = "native",
                sort_mode = "multi",
                column_selectable = False,
                row_selectable = "single",
                row_deletable = False,
                selected_columns = [],
                selected_rows = [0],
                page_action = "native",
                page_current = 0,
                page_size = 10,
                style_table = {'height': '370px', 'width': '1000px', 'overflowX': 'scroll'},
                style_as_list_view = True
            ),
            html.Div(
                id='graph-id',
                className='col s12 m6',
            )
        ]
    ),
    html.Br(),
    html.Hr(),
    
    # Container to hold the geolocation map and a section to display selected animal details
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(
            id='map-id',
            className='col s12 m6',
        ),
        html.Div(
            id='info-id',
            className='col s12 m6',
            children=[
                html.Div(id='info-content'),
            ],
            style={'width': '300px', 'padding': '10px'},  # Adjust width and padding as needed
        ),
    ]),
    html.Hr(),
    
    # Container to hold creaters information and logo
    html.Div(
        children=[
            html.B(html.H4('Created by')),
            html.Img(src='data:image/png;base64,{}'.format(encoded_image2.decode()), 
            style={'width': '75px', 'height': 'auto', 'margin-left': '20px'}),
        ],
        style={'flex-grow': '1', 'display': 'flex', 'flex-direction': 'row'}, 
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# callback for filtering of the data table based on the type of training in question
# returns all animals that fit the criteria of the selected type
@app.callback(Output('datatable-id','data'),
              [Input('animal-type-checklist', 'value')])

def update_dashboard(selected_animal_types):
    # variables to hold the dog breeds, sex type, and age limits that are associated with that training type
    # utilizing variables to increase ease of updating in the future
    waterDogBreed = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
    mountWildBreed = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky','Rottweiler']
    disasterBreed = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
    waterDogSex = ['Intact Female']
    mountWildSex = ['Intact Male']
    disasterSex = ['Intact Male']
    waterAgeMin = 26
    waterAgeMax = 156
    mountWildAgeMin = 26
    mountWildAgeMax = 156
    disasterAgeMin = 20
    disasterAgeMax = 300
    
    if 'Reset' in selected_animal_types:
        # If reset checkbox is selected, show all data
        filtered_data = df.to_dict('records')
        
    elif 'Water' in selected_animal_types:
        filtered_data = df[
            (df['breed'].isin(waterDogBreed)) &
            (df['sex_upon_outcome'].isin(waterDogSex)) &
            ((df['age_upon_outcome_in_weeks'] > waterAgeMin) & 
             (df['age_upon_outcome_in_weeks'] < waterAgeMax))
        ].to_dict('records')
                           
    elif 'MountWilderness' in selected_animal_types:
        filtered_data = df[
            (df['breed'].isin(mountWildBreed)) & 
            (df['sex_upon_outcome'].isin(mountWildSex)) & 
            ((df['age_upon_outcome_in_weeks'] > mountWildAgeMin) & 
             (df['age_upon_outcome_in_weeks'] < mountWildAgeMax))
        ].to_dict('records')
                           
    elif 'Disaster' in selected_animal_types:
        filtered_data = df[
            (df['breed'].isin(disasterBreed)) & 
            (df['sex_upon_outcome'].isin(disasterSex)) & 
            ((df['age_upon_outcome_in_weeks'] > disasterAgeMin) & 
             (df['age_upon_outcome_in_weeks'] < disasterAgeMax))
        ].to_dict('records')
                           
    else:
        # show all data types
        filtered_data = df.to_dict('records')
    
    return filtered_data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    dog_train_breeds = [
        'Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland', 'German Shepherd', 
        'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky','Rottweiler', 
        'Doberman Pinscher', 'Golden Retriever', 'Bloodhound'
    ]
    
    if viewData is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    
    dff_dogs = dff[dff['breed'].isin(dog_train_breeds)]
    breed_counts = dff_dogs.groupby(
        ['animal_type', 'breed', 'sex_upon_outcome', 'age_upon_outcome']).size().reset_index(name='Count')
    
    return [
        dcc.Graph(            
            figure=px.sunburst(
                breed_counts,
                path=['animal_type', 'breed', 'sex_upon_outcome', 'age_upon_outcome'],  # Define the levels
                values='Count',
                title='Number and type of Dogs available'
            )
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# callback that will display specific information about the selected row of the data table
# allows user to quickly and efficiently review details about the animal without having to
# navigate the data table
@app.callback(
    Output('info-content', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def display_data(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    row_index = index[0]  # Get the index of the selected row

    selected_row = dff.iloc[row_index]

    return [
        html.Div([
            html.P(f"Type of Animal - {selected_row['animal_type']}"),
            html.Hr(),
            html.P(f"Name: {selected_row['name']}"),
            html.P(f"Breed: {selected_row['breed']}"),
            html.P(f"Sex: {selected_row['sex_upon_outcome']}"),
            html.P(f"Age: {selected_row['age_upon_outcome']}"),
            html.P(f"Color: {selected_row['color']}"),
        ])
    ]
        
app.run_server(debug=True)


Successfully Connected to Database
Dash app running on http://127.0.0.1:10854/


/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/anaconda/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be remo